In [2]:
__author__ = "Zafar Hussain (University of Helsinki, IVVES Project), Marcin Kowiel (F-Secure)"

import csv
from collections import Counter
import logging
import os
import re
import networkx as nx
import numpy as np
from difflib import SequenceMatcher

logger = logging.getLogger(__name__)


def similar(a, b):
    return round(SequenceMatcher(None, a, b).ratio(), 3)


def draw_tree_multiple_command(tags, token_ids, tokens):
    """
    We are calling this function when there is more than one COMM in the command.
    nodes: tokens
    labels: the labels we assigned to each token
    We extracted the indices of COMM, and then while going over each index, if the index is in the extracted
    indices, that index is marked as COMM, and the subsequent tokens are connected with this node.
    We are labeling each edge with the label, e.g. COMM, SUBCOMM, FLAG etc.
    """
    G = nx.DiGraph()

    if len(tags) == 0:
        return G
    elif len(tags) == 1:
        G.add_edge(token_ids[0], token_ids[0], label=tags[0], end_token=tokens[0])
        return G

    comm_indices = [i_tag for i_tag, tag in enumerate(tags) if tag == "COMM"]
    j = 0
    last_cmd = token_ids[0]

    for i in range(0, len(tags)):
        if i == comm_indices[j]:
            G.add_edge(last_cmd, token_ids[comm_indices[j]], label=tags[comm_indices[j]], end_token=tokens[0])
            last_cmd = token_ids[comm_indices[j]]
            # if not the last command
            if j + 1 < len(comm_indices):
                j += 1

        if tags[i] in ("FLAG", "FLAG_WITH_FLAG_VALUE", "PARAM", "OPERATOR", "SUBCOMM", "COMM_SCRIPT", "SCRIPT"):
            G.add_edge(last_cmd, token_ids[i], label=tags[i], end_token=tokens[i])
        elif tags[i] in ("FLAG_VALUE", "FLAG_SEPARATOR"):
            G.add_edge(token_ids[i - 1], token_ids[i], label=tags[i], end_token=tokens[i])
        elif tags[i] == "OPERATOR" and tags[i + 1] == "OPERATOR_PARAM":
            G.add_edge(last_cmd, token_ids[i], label=tags[i], end_token=tokens[i])
            G.add_edge(token_ids[i], token_ids[i + 1], label=tags[i + 1], end_token=tokens[i + 1])

    return G


def program_name_win(command):
    """
    Get program name for windows platform
    return program name without extension, for example "cmd"  for "C:\\windows\\sys\\cmd.exe"
    """
    return command.split("\\")[-1].split(".")[0].replace('"', "").replace("'", "")


def get_token(token_id):
    return token_id[2]


def get_tokens(token_ids):
    return [token_id[2] for token_id in token_ids]


def token_similarity(edge1, edge2):
    return len(set(get_tokens(edge1)) & set(get_tokens(edge2))) / float(
        len(set(get_tokens(edge1)) | set(get_tokens(edge2)))
    )


def verify_combinations(U, gr_dt_1, gr_dt_2, similarity_combinations, commands_names, commands_distance_matrix):
    """
    U: final graph
    gr_dt_1: dictionary of tokens and labels of command one
    gr_dt_2: dictionary of tokens and labels of command two
    similarity_combinations: this is a reference table, where we have 54 combinations and their class
    """
    U_edges = list(U.edges.data())
    comm_value, subcomm_value, flag_value, param_value = 0, -1, -1, -1
    result = []
    unique_edges = []

    comm_1_edges = gr_dt_1.keys()
    comm_2_edges = gr_dt_2.keys()
    for new_edge in U_edges:
        edge_name = new_edge[2]["label"]
        if edge_name not in unique_edges:
            unique_edges.append(edge_name)
    if unique_edges[0] != "COMM":
        comm_index = unique_edges.index("COMM")
        unique_edges[0], unique_edges[comm_index] = "COMM", unique_edges[0]
    if "SUBCOMM" in unique_edges and unique_edges[1] != "SUBCOMM":
        subcomm_index = unique_edges.index("SUBCOMM")
        unique_edges[1], unique_edges[subcomm_index] = "SUBCOMM", unique_edges[1]

    for edge_name in unique_edges:
        if edge_name == "COMM":
            # not similar
            comm_value = 0

            comm_1 = gr_dt_1["COMM"]
            comm_2 = gr_dt_2["COMM"]
            final_index_1 = max(index for index, item in enumerate(comm_1))
            final_index_2 = max(index for index, item in enumerate(comm_2))
            # if identical
            if comm_1[final_index_1] == comm_2[final_index_2]:
                logger.debug("COMM same")
                comm_value = 1
            else:
                base_cmd_sliced = program_name_win(get_token(comm_1[final_index_1]))
                new_cmd_sliced = program_name_win(get_token(comm_2[final_index_2]))
                # if program names the same but path different
                if base_cmd_sliced == new_cmd_sliced:
                    logger.debug("COMM same")
                    comm_value = 1
                # if close names
                elif (
                    # file names similar
                    similar(base_cmd_sliced, new_cmd_sliced) >= 0.95
                    # and paths similar
                    and similar(get_token(comm_1[final_index_1]), get_token(comm_2[final_index_2])) >= 0.95
                ):
                    comm_value = 1
                # look for aliases
                elif (
                    commands_names is not None
                    and base_cmd_sliced in commands_names
                    and new_cmd_sliced in commands_names
                ):
                    command_row = np.where(commands_names == base_cmd_sliced)[0][0]
                    command_col = np.where(commands_names == new_cmd_sliced)[0][0]
                    sim_score_ref = round(commands_distance_matrix[command_row][command_col], 4)
                    if sim_score_ref > 0.80:
                        comm_value = 1

            if comm_value == 1:
                logger.debug("COMM same")
            else:
                logger.debug("COMM not same")
        elif edge_name == "SUBCOMM":
            print("SUBCOMM", edge_name in comm_1_edges and edge_name in comm_2_edges)
            subcomm_value = 0
            # if missing
            if edge_name not in comm_1_edges and edge_name not in comm_2_edges:
                logger.debug("SUBCOMM missing")
                subcomm_value = -1

            if edge_name in comm_1_edges and edge_name in comm_2_edges:
                # if very close
                res_subcomm = token_similarity(gr_dt_1[edge_name], gr_dt_2[edge_name])
                print(res_subcomm, gr_dt_1[edge_name], gr_dt_2[edge_name])
                if res_subcomm > 0.99:
                    logger.debug("SUBCOMM same")
                    subcomm_value = 1

            if subcomm_value == 0:
                logger.debug("SUBCOMM not same")

        elif edge_name == "FLAG":
            flag_value = 0
            if edge_name not in comm_1_edges and edge_name not in comm_2_edges:
                logger.debug("FLAG missing")
                flag_value = -1

            if edge_name in comm_1_edges and edge_name in comm_2_edges:
                res_flags = token_similarity(gr_dt_1[edge_name], gr_dt_2[edge_name])
                if res_flags > 0.89:
                    logger.debug("FLAG same")
                    flag_value = 1

            if flag_value == 0:
                logger.debug("FLAG not same")

        elif edge_name == "PARAM":
            param_value = 0
            print("PARAM")
            if edge_name not in comm_1_edges and edge_name not in comm_2_edges:
                logger.debug("PARAM missing")
                param_value = -1

            if edge_name in comm_1_edges and edge_name in comm_2_edges:
                if comm_value == 1 or subcomm_value == 1:
                    param_score = 0
                    param_1 = get_tokens(gr_dt_1["PARAM"])
                    param_2 = get_tokens(gr_dt_2["PARAM"])
                    print(param_1, param_2)
                    for z in range(max(len(param_1), len(param_2))):
                        if z < len(param_1) and z < len(param_2):
                            if param_1[z] == param_2[z] or similar(param_1[z], param_2[z]) >= 0.75:
                                param_score += 1
                            elif param_1[z].isnumeric() and param_2[z].isnumeric():
                                param_score += 1

                    if param_score / max(len(param_1), len(param_2)) > 0.65:
                        logger.debug("PARAM same")
                        param_value = 1
                else:
                    res_param = token_similarity(gr_dt_1[edge_name], gr_dt_2[edge_name])
                    print(gr_dt_1[edge_name], gr_dt_2[edge_name], res_param)
                    if res_param > 0.65:
                        logger.debug("PARAM same")
                        param_value = 1

            if param_value == 0:
                logger.debug("PARAM not same")

    result.append([comm_value, subcomm_value, flag_value, param_value])
    result = tuple([item for sublist in result for item in sublist])
    key_names = ["COMM", "SUBCOMM", "FLAG", "PARAM"]
    logger.info([(r, name) for r, name in zip(result, key_names)])
    print([(r, name) for r, name in zip(result, key_names)])
    class_output = similarity_combinations[result]
    return class_output


def create_dict_nodes_labels(tags, tokens):
    """
    Given nodes and labels, we create a dictionary where labels (COMM, SUBCOMM, FLAG etc.) are
    keys and tokens are the values of these keys.
    """
    grouped_data = dict()
    for tag, token in zip(tags, tokens):
        if tag not in grouped_data:
            grouped_data[tag] = []
        grouped_data[tag].append(token)
    return grouped_data


def token_with_tags(tags, tokens):
    return [(tag, i_tag, token) for i_tag, tag, token in zip(range(len(tags)), tags, tokens)]


def normalize_digits(tokens):
    return [re.sub("\d", "0", token) for token in tokens]


def normalize_uuid(tokens):
    return [
        re.sub(
            "[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[1-5][0-9a-fA-F]{3}-[89abAB][0-9a-fA-F]{3}-[0-9a-fA-F]{12}",
            "<<UUID>>",
            token,
        )
        for token in tokens
    ]


def compare_graphs(tags1, tokens1, tags2, tokens2, similarity_combinations, commands_names, commands_distance_matrix):
    """
    Compare two command lines. Works only with windows platform.

    tags1 and tokens1 represent the base_command (command one)
    tags2 and tokens2 represent the new command which is being compared with the base command
    similarity_combinations: this is a reference table, where we have 54 combinations and their class
    For now we have faced one instance where COMM and FLAG had the same values, such as pip. So
    we are marking COMM 'pip' as 'PIP' and FLAG 'pip' remains 'pip'.
    """

    tokens_wth_prefix1 = token_with_tags(tags1, normalize_uuid(normalize_digits(tokens1)))
    tokens_wth_prefix2 = token_with_tags(tags2, normalize_uuid(normalize_digits(tokens2)))

    if "COMM" not in set(tags1) or "COMM" not in set(tags2):
        raise Exception("'COMM' tag not detected")

    g1 = draw_tree_multiple_command(tags1, tokens_wth_prefix1, tokens1)
    g2 = draw_tree_multiple_command(tags2, tokens_wth_prefix2, tokens2)

    # Edges and nodes of both the generated trees are extracted here
    g1_ed = list(g1.edges.data())
    g2_ed = list(g2.edges.data())
    g2_nd = list(g2.nodes)
    g1_nd = list(g1.nodes)

    # A new graph is created by uniting edges and nodes of both the trees.
    combined_graph = nx.DiGraph()
    combined_graph.add_edges_from(g1_ed + g2_ed)
    combined_graph.add_nodes_from(g1_nd + g2_nd)

    # Calling the function with nodes and labels to create dictionaries
    gr_dt_1 = create_dict_nodes_labels(tags1, tokens_wth_prefix1)
    gr_dt_2 = create_dict_nodes_labels(tags2, tokens_wth_prefix2)

    # Classifying the given two commands.
    class_output = verify_combinations(
        combined_graph, gr_dt_1, gr_dt_2, similarity_combinations, commands_names, commands_distance_matrix
    )

    return " ".join(tokens1), " ".join(tokens2), class_output


def load_similarity_config(similarity_definition_path=os.path.join(os.getcwd(), "data", "similarity.csv")):
    with open(similarity_definition_path) as csvfile:
        similarity_combinations = csv.reader(csvfile, delimiter=",", quotechar='"')
        similarity_combinations_dict = {}
        for row in list(similarity_combinations)[1:]:
            is_similar = True if row[4] == "Similar" else False
            similarity_combinations_dict[(int(row[0]), int(row[1]), int(row[2]), int(row[3]))] = is_similar
        return similarity_combinations_dict


def load_similarity_matrix(win_cmd_similarity_matrix=os.path.join(os.getcwd(), "data", "windows_desc_sim_df.csv")):
    with open(win_cmd_similarity_matrix) as csvfile:
        wind_comd_sim_ref = csv.reader(csvfile, delimiter=",", quotechar='"')
        wind_comd_sim_ref = np.array(list(wind_comd_sim_ref))
        wind_comd_sim_ref_columns = wind_comd_sim_ref[0][1:]
        wind_comd_sim_ref = wind_comd_sim_ref[1:, 1:].astype(float)
        return wind_comd_sim_ref_columns, wind_comd_sim_ref
